In [1]:
import glob
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import datetime as dt
import scipy

## Reading Train and Test datasets

In [2]:
train = pd.read_csv('train.csv',sep=',')

In [3]:
len(train)

1710

In [4]:
train.columns

Index(['Last.Name', 'First.Name', 'Age', 'Race', 'Sex', 'State', 'County',
       'Worst.Crime.Display', 'Occurred', 'Convicted', 'Sentence', 'DNA', 'FC',
       'MWID', 'F.MFE', 'P.FA', 'OM', 'ILD', 'CV', 'IO', 'CIU', 'H', 'NC', 'P',
       'F', 'CDC', 'JI', 'M', 'CSH', 'FED', 'A', 'PH', 'SBS', 'CTE'],
      dtype='object')

In [5]:
test = pd.read_csv('test.csv',sep=',')

In [6]:
len(test)

570

In [7]:
test.columns

Index(['Last.Name', 'First.Name', 'Age', 'Race', 'Sex', 'State', 'County',
       'Worst.Crime.Display', 'Occurred', 'Convicted', 'Sentence', 'DNA', 'FC',
       'MWID', 'F.MFE', 'P.FA', 'OM', 'ILD', 'CV', 'IO', 'CIU', 'H', 'NC', 'P',
       'F', 'CDC', 'JI', 'M', 'CSH', 'FED', 'A', 'PH', 'SBS', 'CTE'],
      dtype='object')

## Combining train and test for pre-processing

In [8]:
df = None
train['data_type'] = 'train'
test['data_type'] = 'test'

#df = pd.concat([train,test], axis=1)
df = train

df = df.append(test)

In [9]:
df.head()

,Last.Name,First.Name,Age,Race,Sex,State,County,Worst.Crime.Display,Occurred,Convicted,...,CDC,JI,M,CSH,FED,A,PH,SBS,CTE,data_type
0,Wosu,Joy,33,Black,Female,New York,Erie New York,Child Sex Abuse,1991,1993,...,0,0,0,0,0,0,0,0,16,train
1,Hays,Robert,33,White,Male,Nevada,Clark Nevada,Child Sex Abuse,1992,1993,...,0,0,0,0,0,0,0,0,14,train
2,Bunge,Charles,38,White,Male,New York,Kings New York,"Attempt, Violent",2006,2007,...,0,0,0,0,0,0,0,0,3,train
3,Benitez,Ricardo,54,Hispanic,Male,New York,Queens New York,Robbery,2009,2010,...,0,0,0,0,0,0,0,0,5,train
4,Dixon,Bobby Ray,22,Black,Male,Mississippi,Forrest Mississippi,Murder,1979,1980,...,1,0,0,0,0,0,1,0,30,train


In [10]:
df.rename(columns={'Worst.Crime.Display':'WorstCrimeDisplay'}, inplace=True)

## Checking levels of categorical variables

### Race

In [11]:
df.Race.unique()

array(['Black', 'White', 'Hispanic', 'Asian', 'Native American', 'Other'],
      dtype=object)

### State

#### Observations: Race has only 6 levels and can be easily one-hot-encoded

In [12]:
len(df.State.unique())

82

#### Observations: Too many levels. Even with recursive feature elimination this will be a very costly variable to process. Hence considering less frequent levels into higher frequency levels

In [13]:
df.State.value_counts()

Texas                   346
New York                252
Illinois                221
California              191
Michigan                 87
Ohio                     65
Pennsylvania             64
Florida                  64
Massachusetts            63
North Carolina           60
Wisconsin                54
Louisiana                53
Virginia                 51
Washington               48
Missouri                 46
New Jersey               37
Georgia                  35
Oklahoma                 35
Indiana                  35
Alabama                  27
Maryland                 27
Connecticut              23
Tennessee                21
Arizona                  20
Oregon                   19
Fed-NY                   18
Mississippi              18
Utah                     16
District of Columbia     16
Iowa                     14
                       ... 
Fed-TX                    5
Fed-PA                    5
North Dakota              4
Wyoming                   4
South Dakota        

### Combining all Federal state values into a single 'Federal' level to preserve potential policy compliance

In [14]:
df.loc[df.State.str.startswith('Fed-', na=False),'State'] = 'Federal'

In [15]:
df.State[df.State.str.startswith('Fed-', na=False)]

Series([], Name: State, dtype: object)

### Combining all state levels with < 47 counts into level 'Other' to keep the level count at 10

In [16]:
state_counts =  df.State.value_counts().reset_index()
state_counts.columns=['State','Count']
state_counts.head()

,State,Count
0,Texas,346
1,New York,252
2,Illinois,221
3,California,191
4,Federal,109


In [17]:
state_with_less_counts = state_counts[state_counts.Count<47]['State']
state_with_less_counts

15                Missouri
16              New Jersey
17                Oklahoma
18                 Indiana
19                 Georgia
20                Maryland
21                 Alabama
22             Connecticut
23               Tennessee
24                 Arizona
25                  Oregon
26             Mississippi
27    District of Columbia
28                    Utah
29               Minnesota
30                 Montana
31                Kentucky
32                    Iowa
33                  Nevada
34           West Virginia
35                  Kansas
36                Nebraska
37                  Alaska
38                Colorado
39                Arkansas
40              New Mexico
41          South Carolina
42            Rhode Island
43             Puerto Rico
44                   Idaho
45            North Dakota
46            South Dakota
47                 Wyoming
48                  Hawaii
49                   Maine
50                 Vermont
51                Delaware
5

In [18]:
df.loc[df.State.isin(state_with_less_counts),'State'] = 'Other'

In [19]:
len(df.State.unique())

16

#### Observations: State level count reduced to 10

### County

In [20]:
len(df.County.unique())

646

#### Observations: Way too many levels for even feature elimination to remove insignificant levels. Will not consider this categorical variable for linear regression model building

### WorstCrimeDisplay

In [21]:
len(df.WorstCrimeDisplay.unique())

42

#### Observations: Too many levels. To avoid very long feature elimination time, these levels are dropped to 5

In [22]:
df.WorstCrimeDisplay.value_counts()

Murder                           887
Sexual Assault                   320
Child Sex Abuse                  261
Drug Possession or Sale          254
Robbery                          118
Assault                           91
Attempted Murder                  49
Manslaughter                      45
Weapon Possession or Sale         31
Fraud                             31
Arson                             22
Sex Offender Registration         20
Theft                             18
Kidnapping                        15
Conspiracy                        13
Burglary/Unlawful Entry           12
Other Nonviolent Felony            9
Child Abuse                        9
Other Violent Felony               9
Traffic Offense                    8
Other                              7
Tax Evasion/Fraud                  7
Perjury                            5
Attempt, Violent                   4
Immigration                        4
Forgery                            3
Threats                            3
P

In [23]:
WorstCrimeDisplay_counts =  df.WorstCrimeDisplay.value_counts().reset_index()
WorstCrimeDisplay_counts.columns=['WorstCrimeDisplay','Count']
WorstCrimeDisplay_counts.head()

,WorstCrimeDisplay,Count
0,Murder,887
1,Sexual Assault,320
2,Child Sex Abuse,261
3,Drug Possession or Sale,254
4,Robbery,118


In [24]:
WorstCrimeDisplay_with_high_counts = WorstCrimeDisplay_counts.head()['WorstCrimeDisplay']
WorstCrimeDisplay_with_high_counts

0                     Murder
1             Sexual Assault
2            Child Sex Abuse
3    Drug Possession or Sale
4                    Robbery
Name: WorstCrimeDisplay, dtype: object

In [25]:
df.loc[~df.WorstCrimeDisplay.isin(WorstCrimeDisplay_with_high_counts),'WorstCrimeDisplay'] = 'Other'

In [26]:
len(df.WorstCrimeDisplay.unique())

6

### Sentence

In [27]:
df.Sentence.unique()

array(['25 years', 'Life', '6 years', '22 to life', '22 to 32 years',
       '41 to Life', '3 years and 7 months', 'Death', '18 years',
       '5 to 10 years', 'Life without parole', '30 years', '9 to Life',
       '48 years', '12 years', '13 years', '5 years', '25 to 75 years',
       '20 years', '75 years', 'Not sentenced', '25 to Life', '40 years',
       '50 years', '2 months', '16 years', '180 days', '148 years',
       '15 years', '25 to life', '10 to life', '6 months to 5 years',
       '8 years', '3 years and 6 months', '6 months', '7 years',
       'Probation', '60 years', '10 years', '12 to 20 years',
       '15 to life', '14 years', '30 to life', '22 years',
       '12 to 16 years', '120 days', '26 years', '3 years', '5 to Life',
       '30 to 55 years', '19 to 57 years', '90 days', '24 years',
       '8 months', '7 to 14 years', '35 years', '50 to life',
       '60 to 90 years', '12 to 18 years', '20 to 50 years',
       '8 1/3 to 25 years', '3 to 6 years', '100 years', '2 

In [28]:
len(df.Sentence.unique())

370

In [29]:
df.shape

(2280, 35)

#### Observations: Too many levels. And it is very hard to logically convert the observed values to numericals. Hence converting levels with less # of observations into 'Other' level

In [30]:
df.Sentence.value_counts()

Life                      389
Life without parole       130
Death                     120
Not sentenced             106
Probation                  98
20 years                   48
25 to life                 47
10 years                   45
30 years                   39
15 years                   38
25 years                   37
5 years                    36
25 to Life                 33
6 months                   31
40 years                   30
50 years                   30
12 years                   30
2 years                    29
60 years                   27
3 years                    25
35 years                   23
4 years                    20
7 years                    19
30 days                    19
15 to Life                 18
45 years                   18
8 years                    17
55 years                   16
20 to Life                 16
6 years                    16
                         ... 
17 1/2 years                1
12 to 15 years              1
14 to 35 y

### Reading Sentence levels with highest counts

In [31]:
sentence_counts = df.Sentence.value_counts().reset_index()
sentence_counts.columns = ['Sentence','Count']
sentences_with_high_counts = sentence_counts.head()['Sentence']
sentences_with_high_counts

0                   Life
1    Life without parole
2                  Death
3          Not sentenced
4              Probation
Name: Sentence, dtype: object

### Updating Sentence levels with less counts to level 'Other'

In [32]:
df.loc[~df.Sentence.isin(sentences_with_high_counts),'Sentence'] = 'Other'

In [33]:
len(df.Sentence.unique())

6

### Occured and Convicted

#### Though these two variables are ordinal, for the sake of this analysis they are considered as numeric to avoid costly feature elimination times

## One-hot-encoding

#### Dropping 'Sentence' feature as feature elimination is facing memory constraints. Assumption here is that 'Sentence' which is an independent outcome that has no impact on Exoneration process as much as other features.

In [34]:
categorical_features = ['Race', 'Sex', 'State', 'WorstCrimeDisplay']

#### dummy coding all categorical variables considered for modeling

In [35]:
for cf in categorical_features:
    d = pd.get_dummies(df[cf], prefix=cf,prefix_sep='_')
    df = df.drop(cf,axis=1)
    df = df.join(d)

In [36]:
df.columns

Index(['Last.Name', 'First.Name', 'Age', 'County', 'Occurred', 'Convicted',
       'Sentence', 'DNA', 'FC', 'MWID', 'F.MFE', 'P.FA', 'OM', 'ILD', 'CV',
       'IO', 'CIU', 'H', 'NC', 'P', 'F', 'CDC', 'JI', 'M', 'CSH', 'FED', 'A',
       'PH', 'SBS', 'CTE', 'data_type', 'Race_Asian', 'Race_Black',
       'Race_Hispanic', 'Race_Native American', 'Race_Other', 'Race_White',
       'Sex_Female', 'Sex_Male', 'State_California', 'State_Federal',
       'State_Florida', 'State_Illinois', 'State_Louisiana',
       'State_Massachusetts', 'State_Michigan', 'State_New York',
       'State_North Carolina', 'State_Ohio', 'State_Other',
       'State_Pennsylvania', 'State_Texas', 'State_Virginia',
       'State_Washington', 'State_Wisconsin',
       'WorstCrimeDisplay_Child Sex Abuse',
       'WorstCrimeDisplay_Drug Possession or Sale', 'WorstCrimeDisplay_Murder',
       'WorstCrimeDisplay_Other', 'WorstCrimeDisplay_Robbery',
       'WorstCrimeDisplay_Sexual Assault'],
      dtype='object')

### Defining regression_features - features to be used in feature elimination and model building

#### Dropping 'Sentence' feature as feature elimination is facing memory constraints. Assumption here is that 'Sentence' which is an independent outcome that has no impact on Exoneration process as much as other features.

In [37]:
regression_features = ['Age', 'Occurred', 'Convicted', 'DNA', 'FC', 'MWID', 'F.MFE', 'P.FA', 'OM', 'ILD', 'CV',
       'IO', 'CIU', 'H', 'NC', 'P', 'F', 'CDC', 'JI', 'M', 'CSH', 'FED', 'A',
       'PH', 'SBS', 'Race_Asian', 'Race_Black',
       'Race_Hispanic', 'Race_Native American', 'Race_Other', 'Race_White',
       'Sex_Female', 'Sex_Male', 'State_California', 'State_Federal',
       'State_Florida', 'State_Illinois', 'State_Louisiana',
       'State_Massachusetts', 'State_Michigan', 'State_New York',
       'State_North Carolina', 'State_Ohio', 'State_Other',
       'State_Pennsylvania', 'State_Texas', 'State_Virginia',
       'State_Washington', 'State_Wisconsin',
       'WorstCrimeDisplay_Child Sex Abuse',
       'WorstCrimeDisplay_Drug Possession or Sale', 'WorstCrimeDisplay_Murder',
       'WorstCrimeDisplay_Other', 'WorstCrimeDisplay_Robbery',
       'WorstCrimeDisplay_Sexual Assault']

### Splitting processed data into train and test sets

In [38]:
# train = df[df.data_type=='train']
# test = df[df.data_type=='test']

## Linear Regression

In [39]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import cross_validate

## Helper Functions

#### Runs a k-fold CV on passed datasets

In [40]:
def run_cv(lm, X, y, k):
    return cross_validate(lm, X, y, cv=k,return_train_score=True,return_estimator=True)

#### Executes Linear Regression CV and prints best estimator MSE

In [41]:
def execute_lm_cv(X, y, k):
    cv_results =  run_cv(LinearRegression(), X, y, k)
    print('Train R2:')
    print(cv_results['train_score'])
    print('\nTest R2:')
    print(cv_results['test_score'])
    print('\nBest estimator index based on Train R2:')
    i = list(cv_results['train_score']).index(max(cv_results['train_score']))
    print(i)
    print('\nMSE: {0}'.format(mean_squared_error(y, cv_results['estimator'][i].predict(X))))

### RUN 1 - 2 fold CV on all features - to accommodate for memory constraints

In [42]:
cv_results =  run_cv(LinearRegression(), df[regression_features], df.CTE, 2)

In [43]:
cv_results['test_score']

array([0.59710605, 0.62112091])

In [44]:
cv_results['train_score']

array([0.67395126, 0.64780715])

In [45]:
list(cv_results['train_score']).index(max(cv_results['train_score']))

0

### RUN 2 - k-fold CV - k=2

#### Removing WorstCrimeDisplay features

In [46]:
regression_features_2 = [x for x in regression_features if ((x.startswith('WorstCrimeDisplay') == False) & (x.startswith('State') == False))]
regression_features_2

['Age',
 'Occurred',
 'Convicted',
 'DNA',
 'FC',
 'MWID',
 'F.MFE',
 'P.FA',
 'OM',
 'ILD',
 'CV',
 'IO',
 'CIU',
 'H',
 'NC',
 'P',
 'F',
 'CDC',
 'JI',
 'M',
 'CSH',
 'FED',
 'A',
 'PH',
 'SBS',
 'Race_Asian',
 'Race_Black',
 'Race_Hispanic',
 'Race_Native American',
 'Race_Other',
 'Race_White',
 'Sex_Female',
 'Sex_Male']

In [47]:
execute_lm_cv(df[regression_features_2],df.CTE,2)

Train R2:
[0.66174408 0.63951055]

Test R2:
[0.60127116 0.61952884]

Best estimator index based on Train R2:
0

MSE: 30.834174919743845


#### Removing Race features also

In [55]:
regression_features_3 = [x for x in regression_features if ((x.startswith('WorstCrimeDisplay') == False) & (x.startswith('State') == False) & (x.startswith('Race') == False))]
regression_features_3

['Age',
 'Occurred',
 'Convicted',
 'DNA',
 'FC',
 'MWID',
 'F.MFE',
 'P.FA',
 'OM',
 'ILD',
 'CV',
 'IO',
 'CIU',
 'H',
 'NC',
 'P',
 'F',
 'CDC',
 'JI',
 'M',
 'CSH',
 'FED',
 'A',
 'PH',
 'SBS',
 'Sex_Female',
 'Sex_Male']

In [56]:
execute_lm_cv(df[regression_features_3],df.CTE,2)

Train R2:
[0.65855348 0.63620129]

Test R2:
[0.60086945 0.62109183]

Best estimator index based on Train R2:
0

MSE: 30.983626867116147
